In [ ]:
import pandas as pd 
import os
from sklearn.utils import shuffle
import re 
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers

限制显存大小

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
assert len(gpus) > 0

try:
    # Currently, memory growth needs to be the same across GPUs
    tf.config.experimental.set_memory_growth(gpus[0], True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
except :
    # Memory growth must be set before GPUs have been initialized
    print(e)

载入数据集

In [ ]:
text = []
sentiment = []

# 载入train
file_path = "../data/aclImdb/train"
for file in os.listdir(os.path.join(file_path, 'neg')):
    path = os.path.join(file_path, 'neg', file)
    with open(path) as f:
        text.append(f.readline())
sentiment = [0 for _ in os.listdir(os.path.join(file_path, 'neg'))]

for file in os.listdir(os.path.join(file_path, 'pos')):
    path = os.path.join(file_path, 'pos', file)
    with open(path) as f:
        text.append(f.readline())
pos_sentiment = [1 for _ in os.listdir(os.path.join(file_path, 'pos'))]
sentiment.extend(pos_sentiment)

# 载入test
text_test = []
sentiment_test = []
file_path = "../data/aclImdb/test"
for file in os.listdir(os.path.join(file_path, 'neg')):
    path = os.path.join(file_path, 'neg', file)
    with open(path) as f:
        text_test.append(f.readline())
sentiment_test = [0 for _ in os.listdir(os.path.join(file_path, 'neg'))]

for file in os.listdir(os.path.join(file_path, 'pos')):
    path = os.path.join(file_path, 'pos', file)
    with open(path) as f:
        text_test.append(f.readline())
pos_sentiment_test = [1 for _ in os.listdir(os.path.join(file_path, 'pos'))]
sentiment_test.extend(pos_sentiment_test)

# 合并train、test数据集
text.extend(text_test)
sentiment.extend(sentiment_test)
print(len(text),len(sentiment))

预处理数据

In [ ]:
def preprocess_text(sentence):
    sentence = re.sub('(<)br\s*\/*(>)','',sentence) # Removing <br> as coming in reviews due to html pages
    sentence = re.sub('([?.!])',r' \1 ',sentence) # Putting space between word and punctuation
    sentence = re.sub('[^a-zA-z!.?]+',' ',sentence) # Replacing everythong other that a-zA-z and some punctuation
    sentence = re.sub('(\.\s+)+','. ',sentence)
    sentence = re.sub('(\s+[a-z]\s+)',' ',sentence) # Replacing every single character in between sentence
    sentence = sentence.lower() # lowering all words
    return sentence

processed_text = [preprocess_text(_) for _ in text]
print(len(processed_text), len(sentiment))

参数设置

In [ ]:
max_length = 250
vocab_size = 60000
BATCH_SIZE = 64
EMBEDDING_SIZE = 128
EPOCHS = 32
learning_rate = 0.001
kernel_l2 = 0.0001
recurrent_l2 = 0.0001
activity_l2 = 0.0001

序列化数据

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(processed_text)

processed_text_vector = tokenizer.texts_to_sequences(processed_text)
padded_text_vector = tf.keras.preprocessing.sequence.pad_sequences(processed_text_vector,maxlen=max_length, 
                                                                   truncating='post', padding='post')
print(padded_text_vector.shape)

划分训练集与测试集

In [ ]:
# 训练集，前25000条数据
train_text = padded_text_vector[:25000]
train_sentiment = sentiment[:25000]
train_text, train_sentiment = shuffle(train_text, np.array(train_sentiment), random_state=2)
# 测试集，后25000条数据
test_text = padded_text_vector[25000:]
test_sentiment = sentiment[25000:]
test_text, test_sentiment = shuffle(test_text, np.array(test_sentiment), random_state=3)

print(len(train_text), len(test_text), len(train_sentiment), len(test_sentiment))

划分训练集与验证集

In [ ]:
text_train, text_val, sentiment_train, sentiment_val = train_test_split(train_text,train_sentiment, test_size=0.2,
                                                                       random_state=2)
print(len(text_train), len(text_val), len(sentiment_train), len(sentiment_val))

构建Att-BiLSTM模型

In [ ]:
_input = tf.keras.Input(shape=(max_length,))

token_embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=EMBEDDING_SIZE, input_length=max_length)(_input)

bilstm_layer = tf.keras.layers.Bidirectional(tf.keras.layers.CuDNNLSTM(256, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', 
                                    kernel_regularizer=regularizers.l2(kernel_l2),
                                    recurrent_regularizer=regularizers.l2(recurrent_l2),
                                    activity_regularizer=regularizers.l2(activity_l2),
                                    return_sequences=True), merge_mode='ave')(token_embedding)

query = tf.keras.layers.Dense(256)(bilstm_layer)
value = tf.keras.layers.Dense(256)(bilstm_layer)

attention_output = tf.keras.layers.Attention()([query, value])

output = tf.keras.layers.GlobalAveragePooling1D()(attention_output)
output = tf.keras.layers.Dropout(0.5)(output)
output = tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(kernel_l2))(output)
model = tf.keras.Model(_input, output)
model.summary()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, restore_best_weights=True)

训练模型

In [ ]:
model.fit(x=text_train,y=sentiment_train, batch_size=BATCH_SIZE, epochs=EPOCHS, shuffle=True,
          validation_data=[text_val,sentiment_val], callbacks=[callback])

绘制模型训练结果

In [ ]:
import matplotlib.pyplot as plt
from pylab import *

subplots_adjust(left=0.0,bottom=0.0,top=3,right=2)

plt.subplot(2, 2, 1)
plt.plot(model.history.history['loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title("train loss")

plt.subplot(2, 2, 2)
plt.plot(model.history.history['acc'])
plt.xlabel('epoch')
plt.ylabel('acc')
plt.title("train accuracy")

plt.subplot(2, 2, 3)
plt.plot(model.history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title("validation loss")

plt.subplot(2, 2, 4)
plt.plot(model.history.history['val_acc'])
plt.xlabel('epoch')
plt.ylabel('acc')
plt.title("validation accuracy")

plt.show()

在测试集上评估模型

In [ ]:
model.evaluate(test_text, test_sentiment)

绘制模型图

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file="Att-BiLSTM.png")

模型预测

In [ ]:
x = '''I admit, the great majority of films released before say 1933 are just not for me. Of the dozen or so "major" silents I have viewed, one I loved (The Crowd), and two were very good (The Last Command and City Lights, that latter Chaplin circa 1931).<br /><br />So I was apprehensive about this one, and humor is often difficult to appreciate (uh, enjoy) decades later. I did like the lead actors, but thought little of the film.<br /><br />One intriguing sequence. Early on, the guys are supposed to get "de-loused" and for about three minutes, fully dressed, do some schtick. In the background, perhaps three dozen men pass by, all naked, white and black (WWI ?), and for most, their butts, part or full backside, are shown. Was this an early variation of beefcake courtesy of Howard Hughes?'''
x = preprocess_text(x)
vec = tokenizer.texts_to_sequences([x])
pad_vec = tf.keras.preprocessing.sequence.pad_sequences(vec, maxlen=max_length, truncating='post', padding='post')
result = model.predict(pad_vec)
print(result > 0.5)